In [0]:
from pyspark.sql import functions as F
from delta.tables import *

TABELA_ORIGEM = "v_credit.silver.tb_canal"
TABELA_DESTINO = "v_credit.gold.dm_canal"

### Transformacao Silver → Gold (Dimensao Canal)
Le tb_canal (Silver) e adiciona coluna derivada de negocio:
- **ds_tipo_interacao**: Classificacao de canais
  - Canais com "URA", "CHAT", "BOT" → "Automatizado"
  - Outros → "Humanizado"

Faz MERGE em dm_canal (Gold).

**Importante**: Permite comparar performance Bot vs Humano (tempo de espera, CSAT, custo).

### Configuracao
Define origem (Silver tb_canal) e destino (Gold dm_canal - dimensao do Star Schema).

In [0]:
df_silver = spark.read.table(TABELA_ORIGEM)

df_gold = df_silver.select(
    F.col("cd_canal"),
    F.col("nm_canal"),
    F.col("st_ativo"),

    F.when(
        (F.upper(F.col("nm_canal")).like("%URA%")) | 
        (F.upper(F.col("nm_canal")).like("%CHAT%")) | 
        (F.upper(F.col("nm_canal")).like("%BOT%")), 
        "Automatizado"
    ).otherwise("Humanizado").alias("ds_tipo_interacao")
)

DeltaTable.forName(spark, TABELA_DESTINO).alias("t").merge(
    df_gold.alias("s"), "t.cd_canal = s.cd_canal"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

print(f"✅ Dimensão Canal carregada!")